# Задача C: Нахождение кратчайшего цикла в ориентированном невзвешенном графе

Источник: [лаба про BFS](http://cs.mipt.ru/algo/lessons/sem_2/09.graph_bfs.html).

Дан ориентированный граф.
Вершины пронумерованы от 0.
Трeбуется с помощью обхода в ширину найти цикл минимальной длины.
 
## Формат входных данных

На вход программе в первой строке подаются через пробел два числа: N (2 <= N <= 1000) — число вершин в графе и M (1 <= M <= 20000) — число рёбер.
В следующих M строках задаются рёбра, по два числа в каждой строке — номера соединённых вершин (соответствующее ребро идет из первой вершины во вторую).

## Формат выходных данных

Требуется распечатать номера вершин, задающих минимальный цикл в графе.
Номера вершин нужно вывести в порядке следования по циклу.
Если минимальных циклов несколько вывести любой.
Если циклов нет вывести строку "NO CYCLES" без кавычек. 

## Примеры

Пример 1:
```
Вход:

4 1
3 1

Выход:

NO CYCLES
```

Пример 2:
```
Вход:

8 8
7 3
3 1
5 2
6 4
6 7
4 7
1 6
1 2

Выход:

1 6 7 3
```

## Решение

### Считывание данных ("сырой" граф)

In [1]:
V, E = [int(n) for n in input().split()]

edges = []

for _ in range(E):
    v1, v2 = [int(n) for n in input().split()]
    edges.append((v1, v2))

8 8
7 3
3 1
5 2
6 4
6 7
4 7
1 6
1 2


### "Причёсывание" графа

с прицелом на последующий DFS для поиска цикла

In [2]:
class Node:
    def __init__(self, value):
        self.value = value
        self.descendants = list()
        
        # Проводили ли поиск в ширину из данной вершины (хотя бы раз)
        # (необходимость этого поля под сомнением)
        self.is_bfsed = False

    def __eq__(self, other):
        return self.value == other.value

In [3]:
nodes = [
    Node(i) for i in range(V)
]

for edge in edges:
    v1, v2 = edge
    nodes[v1].descendants.append(nodes[v2])  # пользуемся тем, что вершины пронумерованы

### Поиск цикла

In [4]:
# Можно бы было бэ-эф-эс-ную волну оформить в отдельный класс
# Стало бы проще
# Или нет -- точнее, не во всех аспектах.
# Поэтому отдельный класс оставим только "в проекте"

# class Wave:
#     def __init__(self, start: Node):
#         self.nodes = [start]
    
#     def last(self) -> Node:
#         return self.nodes[-1]
    
#     def append(self, node: Node) -> None:
#         self.nodes.append(node)

In [5]:
def find_cycle_bfs(root):
    wave = [root]
    queue = [wave]
    
    return _find_cycle_bfs(queue)


def _find_cycle_bfs(queue):
    if len(queue) == 0:
        return None
    
    wave = queue.pop(0)  # It's not very nice to pop from list's start, but whatever
    root = wave[-1]

    for d in root.descendants:
        if d.is_bfsed and d in wave:  # цикл
            return restore_cycle(first=d, last=root, wave=wave)
        
        new_wave = list(wave) + [d]
        queue.append(new_wave)
    
    root.is_bfsed = True
    
    return _find_cycle_bfs(queue)


def restore_cycle(first, last, wave):
    assert first in wave
    assert last in wave
    assert wave[-1] == last

    cycle_start_index = wave.index(first)
    cycle = wave[cycle_start_index:]

    return cycle

In [6]:
# Ищет первый цикл! (возможно, есть и ещё меньшей длины...)

for node in nodes:
    if node.is_bfsed:
        continue
    
    cycle = find_cycle_bfs(node)
    
    if cycle is not None:
        cycle_values = [n.value for n in cycle]
        
        print(*cycle_values)
        
        break
else:
    print('NO CYCLES')

1 6 7 3
